In [ ]:
import mne
import os
# os.chdir('C:\\Users\\Esimk_000\\Desktop\\School\\Spring 2018\\Cs59866\\EEG Project\\seniorprojecteeg-master-fork\\preprocessing\\Artifact_Removal')
%run preprocecssing_helpers.ipynb
%run Batch_ArtifactFilter_Epoch.ipynb
%run ../StimCodes.ipynb

import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)
matplotlib.rcParams.update({'font.size': 15})

In [ ]:
# path = 'D:/EEGLAB/'
# eeglab_files = list_eeglab_files(path)

# path = '/sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/'

In [ ]:
# files = os.listdir(path)

In [11]:
modality_lexicality_event_ids = {
   "AALL" : 112,
   "AALN" : 212,
   "AANL" : 312,
   "AANN" : 412,
   "AVLL" : 512,
   "AVNN" : 612,
   "VALL" : 712,
   "VANN" : 812,
   "VVLL" : 912,
   "VVLN" : 1012,
   "VVNL" : 1112,
   "VVNN" : 1212
}

In [1]:
# def get_epoch_trials(epoch_object):
#     #note: third stim-code has a value that is 1 more than the first.
#     events = epoch_object.events
#     found_first_stim_code = False
#     found_second_stim_code = False
#     found_third_stim_code = False
#     current_trial = []
#     trials = []
#     for event in events:
#         if not found_first_stim_code and not found_second_stim_code and not found_third_stim_code: #still searching for the end of trial
#             current_trial.append(event)
#             found_first_stim_code = True
#             first_stim_code = event
#         elif not found_second_stim_code and not found_third_stim_code:
#             current_trial.append(event)
#             found_second_stim_code = True
#         elif not found_third_stim_code and (event[-1] + 1) == first_stim_code[-1]: 
#             current_trial.append(event)
#             found_third_stim_code = True
#             trials.append(current_trial)
#             current_trial = []
#             found_first_stim_code = False
#             found_second_stim_code = False
#             found_third_stim_code = False
#         elif not found_third_stim_code and (event[-1] + 1) != first_stim_code[-1]:
#             # there is no third event, trial is unfinished.
#             current_trial = []
#             current_trial.append(event)#assuming it's first stim-code...for now
#             found_first_stim_code = True
#     return trials

In [ ]:
def get_stim_code_epoch_trials(epoch_object):
    """
    Description:
        returns a numpy 3darray of trials. Checks if three consecutive events follow the format of 
        the first stim-code in the trial is equal to the last plus one. For example, if the first stim
        code is 12, then the last must be 11. If not, the method rejects that trial.
        
    Variables:
        epoch_object:
            mne.epochs object.
            
    returns:
        trials: 3d numpy array
    """
    events = epoch_object.events
    found_third_stim_code = False
    current_trial = []
    trials = []
    
    for i in range(len(events[:,-1])-2):
        found_third_stim_code = False
        while (not found_third_stim_code):
            stim_code_1 = events[i]
            stim_code_2 = events[i+1]
            stim_code_3 = events[i+2]
#             print(stim_code_1,stim_code_2,stim_code_3)
            
            if stim_code_1[-1] == stim_code_3[-1] + 1:
                trials.append([stim_code_1,stim_code_2,stim_code_3])
                found_third_stim_code = True
            else:
                
                break;
    return trials

In [ ]:
def stim_code_combination_from_trial(stim_code_tuple, stim_combinations):
    """
    Description:
        returns the correct stim-code combination event id 
        example - a trial has events (20, 15, 19), this method will match those values to a 
        stim combination that is defined in the dictionary stim_combinations
    """
    for stim_code in stim_combinations.keys():
        if stim_code_tuple == stim_code:
            return stim_combinations[stim_code]
    return "NaN"

In [5]:
def convert_epoch_events_to_stim_combinations(epoch_object):
    trials = get_stim_code_epoch_trials(epoch_object)
#     print(trials)
    new_events_master = []
    
    for trial in trials:
#         print(trial)
        trial = np.array(trial)
        stim_codes = trial[:,-1]
        try:
#             print("stim_codes",stim_codes)
            stim_code_tuple = (stim_codes[0],stim_codes[1], stim_codes[2])
#             print("stim_code_tuple",stim_code_tuple)
            stim_combination = stim_code_combination_from_trial(stim_code_tuple, stim_combinations)
#             print("stim_combination",stim_combination)
            new_event_id = modality_lexicality_event_ids[stim_combination]
#             print("New event_id", new_event_id)
            trial[:,-1] = new_event_id
            new_events_master.append(trial)
        except:
            pass
#             print("Error with stim-code: {0}".format(stim_code_tuple))
#     print(new_events_master)
    return np.concatenate([x for x in new_events_master])

In [6]:
def epoch_indexes_to_drop(epoch_object, new_events):
    """
    Description:
        Some epoch objects have events that are not in order for a trial. 
        These events must be dropped. This method figures out which indicies in the events
        numpy array need to be dropped.
    
    Variables:
        epoch_object:
            mne.epochs object
        new_events:
            3d numpy array of events that have already been converted to stim-code combinations
            use method convert_epoch_events_to_stim_combinations to get this array.
    returns:
    -------
        a list of indices to drop.
    
        
    """
    epoch_events = epoch_object.events
    indices_to_drop = []
    for i in range(len(epoch_events)):
        if epoch_events[i,0] not in new_events[:,0]:
            indices_to_drop.append(i)
    return indices_to_drop

In [ ]:
def generate_indices_tuples(indices_to_drop, len_epochs):
    """
    Description:
        To drop the epochs that correspond to bad events in an epoch object, a list of (start,end)
        tuples are required.
        
    Variables:
        indices_to_drop:
            A list of ints 
            
        len_epochs:
            the number of epochs in the epoch object that will have it's bad epochs dropped.
    returns:
    -------
        if indices_to_drop is empty, return empty list
        else return a list of tuples.
    """
    indices_tuples = []
    if indices_to_drop !=[]:
        
        for i in range(len(indices_to_drop)):
            if i == 0:
                indices_tuples.append((0,indices_to_drop[i]))
                print(indices_tuples)
            else:
                if indices_to_drop[i-1] +1 == indices_to_drop[i]:
                    pass
                else:
                    indices_tuples.append((indices_to_drop[i-1] +1, indices_to_drop[i]))
            print(indices_to_drop)

        indices_tuples.append((indices_to_drop[-1] +1, len_epochs))
        return indices_tuples
    else:
        return []
        
def drop_bad_epoch_events(epoch_object, indices_to_drop):
    """
    Description:
        drops the epochs of an epoch object that correspond to bad events that do not line up
        with a trial. For example: events in this order [4,5,3,2,4,5,3] would require the event labeled
        as "2" to be dropped because it does not line up with the trial [3,5,4].
    
    Variables:
        epoch_object:
            mne.epochs object.
        indices_to_drop:
            a list of ints that have the index of each epoch to drop.
    returns:
    -------
        epoch_object:
            mne.epochs object with the bad epochs droppped.
    
    """
    indices_tuples = generate_indices_tuples(indices_to_drop, epoch.events.shape[0])
    epoch_list = []
    if indices_tuples !=[]:
        for index in indices_tuples:
            epoch_list.append(epoch_object[index[0]:index[1]])
        if epoch_list != []:
            return mne.epochs.concatenate_epochs(epoch_list)
    else: 
        return epoch_object

In [13]:
def convert_epoch_object_events(epoch_object):
    """
    Description:
        Method converts the 1-24 stim-code events of an epoch object to stim-code
        combinations. Given the following sequence of events [4,5,3,11,6,10], they will be 
        considered in groups of three. The first group is (4,5,3) and the second is (11,6,10).
        These two are also considered trials. They are then matched with a dictionary called 
        "stim_combinations" found in ../preprocessing/StimCodes and the new sequence will 
        become [712,712,712,212,212,212]. The groups belong to trails and therefore
        must be given the same label for each item in that trial. 
        
        This method also drops the bad epochs that do not line up with trials.
    Variables:
        epoch_object:
            mne.epochs object
            
    returns:
    --------
        new_epoch:
            an mne.epochs object with events converted to stim_code combinations. 
    """
    new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch_object.copy())
    indices_to_drop = epoch_indexes_to_drop(epoch_object.copy(), new_events)
    new_epoch = drop_bad_epoch_events(epoch_object.copy(), indices_to_drop=indices_to_drop)
    new_epoch.events = new_events
    new_epoch.event_id = modality_lexicality_event_ids
    return new_epoch

In [16]:
# for epoch_file in epoch_files:
#     file = os.path.join(path, epoch_file)
#     epoch = mne.read_epochs(file, preload = True);
    
#     print("**epoch events shape", epoch.events.shape)
# #     try:
        
#     new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch.copy())
#     print("**new_events shape", new_events.shape)

#     indicies_to_drop = epoch_indexes_to_drop(epoch.copy(), new_events)
#     print("**indicies to drop", indicies_to_drop)
#     new_epoch = drop_bad_epoch_events(epoch.copy(), indicies_to_drop=indicies_to_drop)
#     print("**new epoch shape", new_epoch.get_data().shape)
# #     except:
# #         print("**Error with file {0}".format(epoch_file))

In [ ]:
# def isCorrect_stim_code_trials(trials):
#     """
#     Description:
#         A sanity check that makes sure all trials are in the correct order
#         The third stim-code needs to have a value that is one less than the 
#         first stim-code.
        
#     Variables:
#         trials: An ndarray of shape  (n_trials, 3, 3)
#     -----
#     returns:
#         boolean if all are correct, or one is incorrect. 
#         It will print all incorrect trials
#     """
#     has_incorrect_trial = True
#     for trial in trials:
#         if trial[:,-1][0] == trial[:,-1][-1] + 1:
#             pass
#         else:
#             print("Incorrect Trial \n: {0}".format(trial))
#             has_incorrect_trial = False
            
                    
#     return has_incorrect_trial

In [ ]:
# isCorrect_stim_code_trials(epoch_trials)

In [ ]:
# def create_describer_events(epoch, eeglab):
#     """
#     Description:
#         uses raw eeglab events and epoch events to return a new dictionary
#         with key the timestamp of an event, and value is the new event_id.
#         These event_ids can later be split/manipulated to label data by specific 
#         events
#     Variables:
#         epoch: mne.io.epoch object
#         raw_events: an ndarray of 3d integers shape 
        
#     returns:
#         dictionary
#     """
#     raw_events = find_events(eeglab)
#     raw_trials = get_trials(raw_events)
#     epoch_events = epoch.events
#     epoch_trials = get_epoch_trials(epoch)
    
#     for epoch_trial in epoch_trials:
#         for event in epoch_trial
#             if event[0] in 
#     #  